In [54]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [55]:

model = keras.models.load_model('object_detection_model.h5')


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [56]:
class_names = ['Bicycle', 'Boat', 'Bottle', 'Bus', 'Car', 'Cat', 'Chair', 'Cup', 'Dog', 'Motorbike', 'People', 'Table']


video_path = r'C:\Users\Lenovo\Downloads\production_id_4833483 (1080p).mp4'
cap = cv2.VideoCapture(video_path)

In [57]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'H264')
output_path = 'output_video.mp4'  #
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))


In [58]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to match the input size expected by the model
    input_frame = cv2.resize(frame, (224, 224)) / 255.0
    input_frame = np.expand_dims(input_frame, axis=0)

    predictions = model.predict(input_frame)

    for i, pred in enumerate(predictions[0]):
        if pred.max() > 0.5:
            class_index = np.argmax(pred)
            class_label = class_names[class_index]
            confidence = pred[class_index].item() if pred.ndim > 0 else pred.item()

            bbox_color = (0, 255, 0)
            # Draw bounding box on the original frame
            cv2.putText(frame, f'{class_label}: {confidence:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, bbox_color, 2, cv2.LINE_AA)

    out.write(frame)
    cv2.imshow('Object Detection', frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 43ms/step
